<a href="https://colab.research.google.com/github/UmaBalannavar/CNS-ASSINMENT/blob/main/ECC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ecc_ecdsa_demo.py
# Educational ECDSA implementation (secp256k1) for keygen, sign, verify.
# WARNING: This is educational. For production use well-tested libs.

import random
import hashlib

# secp256k1 domain parameters
P = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
A = 0
B = 7
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

# Point at infinity representation
INF = None

def inv_mod(x, p):
    # modular inverse
    return pow(x, p-2, p)

def is_on_curve(point):
    if point is None:
        return True
    x, y = point
    return (y*y - (x*x*x + A*x + B)) % P == 0

def point_add(p1, p2):
    if p1 is None:
        return p2
    if p2 is None:
        return p1
    x1, y1 = p1
    x2, y2 = p2
    if x1 == x2 and (y1 != y2 or y1 == 0):
        return None
    if x1 == x2:
        # point doubling
        m = (3 * x1 * x1 + A) * inv_mod(2 * y1, P) % P
    else:
        m = (y2 - y1) * inv_mod(x2 - x1, P) % P
    x3 = (m*m - x1 - x2) % P
    y3 = (m*(x1 - x3) - y1) % P
    return (x3, y3)

def scalar_mult(k, point):
    # double-and-add
    result = None
    addend = point
    while k:
        if k & 1:
            result = point_add(result, addend)
        addend = point_add(addend, addend)
        k >>= 1
    return result

# ---------- Key generation ----------
def ecc_keygen():
    priv = random.randrange(1, N-1)
    pub = scalar_mult(priv, (Gx, Gy))
    return priv, pub
# ---------- ECDSA sign / verify ----------
def ecc_sign(msg: bytes, priv):
    z = int.from_bytes(hashlib.sha256(msg).digest(), 'big')
    while True:
        k = random.randrange(1, N-1)
        x1y1 = scalar_mult(k, (Gx, Gy))
        if x1y1 is None:
            continue
        r = x1y1[0] % N
        if r == 0:
            continue
        s = (inv_mod(k, N) * (z + r * priv)) % N
        if s == 0:
            continue
        return (r, s)
def ecc_verify(msg: bytes, signature, pub):
    r, s = signature
    if not (1 <= r < N and 1 <= s < N):
        return False
    z = int.from_bytes(hashlib.sha256(msg).digest(), 'big')
    w = inv_mod(s, N)
    u1 = (z * w) % N
    u2 = (r * w) % N
    p = point_add(scalar_mult(u1, (Gx, Gy)), scalar_mult(u2, pub))
    if p is None:
        return False
    return (p[0] % N) == r
# ---------- Example ----------
if __name__ == "__main__":
    print("Generating ECC keypair (secp256k1)...")
    priv, pub = ecc_keygen()
    print("Private key (int):", priv)
    print("Public key (point):", pub)
    message = b'Hello ECC ECDSA!'
    sig = ecc_sign(message, priv)
    print("Signature (r,s):", sig)
    print("Verify:", ecc_verify(message, sig, pub))


Generating ECC keypair (secp256k1)...
Private key (int): 60196523917814974742155434478327838367240126605474613215882754349596949252428
Public key (point): (49479716927223160009354880764119345418734594451546415707158114106889837968043, 8001426918195017072273547036016649772303525928666424330686446193083517308281)
Signature (r,s): (50377278674820566623191098090530776790799047701123453659832587932803153778565, 590617542890027681003843942030028321670525249151758955471152219660622925794)
Verify: True
